In [1]:
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from IPython.display import display 

import model_utils as utils

# Pretty display for notebooks
%matplotlib inline
pd.set_option('display.max_colwidth', -1)

dataset, features, labels = utils.getDataSet("datasets/dataset_under_sampled.csv.gz")

,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,CIE_10,...,PQR_CLASE_SNS,PQR_ESTADO,PQR_TIPOATENCION,PQR_TIPOPETICION,TRIM,AFEC_LOCATION,ENT_LOCATION,PET_LOCATION,COMPLETE_MOTIVE,RIESGO_VIDA
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.125322,...,0.0,0.000000,1.000000,0.000000,0.150914,0.000743,0.000581,0.159041,0.000486,0
1,0.855981,0.530003,1.000000,0.445166,0.290884,0.304503,1.000000,1.0,0.0,0.125322,...,1.0,0.091348,0.512068,0.658277,0.000000,0.342978,0.336340,0.296725,0.316076,0
2,0.487840,0.000000,0.865504,0.000000,0.290884,0.000000,0.661319,1.0,0.0,0.125322,...,1.0,0.091348,0.512068,0.658277,0.150914,0.261972,0.346242,0.228888,0.381179,0
3,0.449334,0.640631,1.000000,0.445166,0.290884,0.304503,0.661319,1.0,0.0,0.125322,...,0.0,0.091348,0.512068,0.000000,1.000000,0.257469,0.338034,0.215414,0.000622,0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.125322,...,0.0,0.000000,1.000000,0.000000,0.205221,0.044997,0.001214,0.159041,0.000000,0


## Shuffle and Split Data

In [3]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'labels' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    labels, 
                                                    test_size = 0.20, 
                                                    random_state = 10)

# Show the results of the split
print "features_final set has {} samples.".format(features.shape[0])
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

features_final set has 577622 samples.
Training set has 462097 samples.
Testing set has 115525 samples.


### Creating a Training and Predicting Pipeline

In [5]:
'''
TP = np.sum(income) # Counting the ones as this is the naive case. Note that 'income' is the 'income_raw' data 
encoded to numerical values done in the data preprocessing step.
FP = income.count() - TP # Specific to the naive case

TN = 0 # No predicted negatives in the naive case
FN = 0 # No predicted negatives in the naive case
'''

tp = float(np.sum(labels['RIESGO_VIDA']))
fp = float(labels['RIESGO_VIDA'].count() - tp)
tn = 0
fn = 0

# TODO: Calculate accuracy, precision and recall
recall = tp / (tp + fn)
precision = tp / (tp + fp)

# TODO: Calculate F-score using the formula above for beta = 0.5 and correct values for precision and recall.
# HINT: The formula above can be written as (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)
beta = 2
fscore = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

# Print the results 
print "Naive Predictor: [F-score: {:.4f}]".format(fscore)

Naive Predictor: [F-score: 0.8333]


In [6]:
import visuals as vs
from sklearn.metrics import fbeta_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
# Initialize the three models
clf_A = SGDClassifier(random_state = 300)
clf_B = RandomForestClassifier(random_state = 300)
clf_C = AdaBoostClassifier(random_state = 300)


samples_100 = len(y_train)

# Collect results on the learners
dfResults = pd.DataFrame(columns=['learner', 'learner_index', 'size_index', 'train_time', 'pred_time', 'f_test', 'f_train'])

for k, clf in enumerate([clf_A, clf_B, clf_C]):
    clf_name = clf.__class__.__name__    
    dfResults = utils.train_predict(clf, k, 0, samples_100, X_train, y_train, X_test, y_test, dfResults)


/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SGDClassifier trained on 462097 samples.


/anaconda2/lib/python2.7/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
model_utils.py:35: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  learner = learner.fit(X_train[:sample_size], y_train[:sample_size])


RandomForestClassifier trained on 462097 samples.
AdaBoostClassifier trained on 462097 samples.


In [7]:
display( dfResults.sort_values(by=['f_test'], ascending = False)[['learner', 'f_test']])

,learner,f_test
0,SGDClassifier,0.895014
1,RandomForestClassifier,0.885759
2,AdaBoostClassifier,0.882964


# Tuning Models

## Tuning RandomForestClassifier

In [8]:
from sklearn.externals import joblib

rfClassifier = RandomForestClassifier(random_state = 20)

rfParameters = {    
  'criterion':['gini', 'entropy'],
  'max_depth':[5, 10],
  'max_features':['auto', 'sqrt', 'log2', None],
  'class_weight': ['balanced', 'balanced_subsample'], 
}

rfClassifier = utils.tuneClassifier(rfClassifier, rfParameters, X_train, X_test, y_train, y_test)

joblib.dump(rfClassifier, 'rfClassifier.joblib') 

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Unoptimized model
------
F-score on testing data: 0.8853

Optimized Model
------
Final F-score on the testing data: 0.9131


['rfClassifier.joblib']

## Tuning SGDClassifier

In [9]:
sdgClassifier = SGDClassifier(random_state = 20)

sdgParameters = {
  'eta0':[0.01],
  'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
  'penalty':['none', 'l2', 'l1', 'elasticnet'],
  'learning_rate':['constant', 'optimal', 'invscaling'],
  'class_weight' :['balanced']
  #'max_features':['auto', 'sqrt', 'log2', None],
}

sdgClassifier = utils.tuneClassifier(sdgClassifier, sdgParameters, X_train, X_test, y_train, y_test)

joblib.dump(rfClassifier, 'sdgClassifier.joblib') 

Unoptimized model
------
F-score on testing data: 0.8886

Optimized Model
------
Final F-score on the testing data: 0.8821


['sdgClassifier.joblib']

## Tuning AdaBoostClassifier

In [12]:
from sklearn.ensemble import AdaBoostClassifier
adaClassifier = AdaBoostClassifier(random_state = 20)

adaParameters = {
  'learning_rate':[0.1, 0.5, 1],
  'algorithm' :['SAMME', 'SAMME.R']
  #'max_features':['auto', 'sqrt', 'log2', None],
}

sdgClassifier = utils.tuneClassifier(adaClassifier, adaParameters, X_train, X_test, y_train, y_test)

joblib.dump(rfClassifier, 'adaClassifier.joblib') 

/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:1943: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Unoptimized model
------
F-score on testing data: 0.8830

Optimized Model
------
Final F-score on the testing data: 0.9108


['adaClassifier.joblib']

## Testing Model with excluded records

In [9]:
dataset_excluded, features_excluded, labels_excluded = utils.getDataSet("datasets/dataset_rus.csv.gz")

predictions_excluded = rfClassifier.predict(features_excluded)
f_test_excluded =  fbeta_score(labels_excluded, predictions_excluded, 2)
print(f_test_excluded)

,AFEC_EDADR,AFEC_EDUC,AFEC_GENERO,AFEC_GETNICO,AFEC_PARENTESCO,AFEC_POBESPECIAL,AFEC_REGAFILIACION,AFEC_TIPOPER,ALTO_COSTO,CIE_10,...,PQR_CLASE_SNS,PQR_ESTADO,PQR_TIPOATENCION,PQR_TIPOPETICION,TRIM,AFEC_LOCATION,ENT_LOCATION,PET_LOCATION,COMPLETE_MOTIVE,RIESGO_VIDA
0,0.862194,0.530003,0.865504,0.445166,0.478849,0.304503,1.000000,1.0,0.0,0.086716,...,1.0,0.091348,0.512068,0.658277,0.150914,0.230700,0.280929,0.278713,0.328084,0
1,0.862194,0.640631,1.000000,0.445166,0.565950,0.304503,0.661319,1.0,0.0,0.469485,...,1.0,0.091348,1.000000,0.658277,0.150914,0.303620,0.343342,0.307024,0.298898,0
2,0.862194,1.000000,1.000000,0.445166,0.549692,0.304503,0.661319,1.0,0.0,0.023123,...,1.0,0.091348,1.000000,0.658277,0.150914,0.420555,0.531137,0.399282,0.143884,0
3,0.487840,0.858045,0.865504,0.445166,0.459473,0.304503,0.661319,1.0,0.0,0.125322,...,1.0,0.091348,1.000000,0.658277,0.150914,0.212932,0.251380,0.179645,0.209656,0
4,0.862194,0.640631,1.000000,0.445166,0.549692,0.304503,0.661319,1.0,0.0,0.152003,...,1.0,0.091348,1.000000,0.658277,0.150914,0.341353,0.460581,0.291511,0.316076,0


0.0


In [8]:
print(dataset.equals(dataset_excluded))

False
